In [1]:
from vllm import LLM, SamplingParams

In [2]:
llm = LLM("Qwen/Qwen2.5-Math-1.5B-Instruct")

INFO 12-05 21:14:31 config.py:350] This model supports multiple tasks: {'generate', 'embedding'}. Defaulting to 'generate'.
INFO 12-05 21:14:31 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='Qwen/Qwen2.5-Math-1.5B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-Math-1.5B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=Qwen/Qwen2.5-Mat

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 12-05 21:14:37 model_runner.py:1077] Loading model weights took 2.8797 GB
INFO 12-05 21:14:38 worker.py:232] Memory profiling results: total_gpu_memory=11.64GiB initial_memory_usage=3.91GiB peak_torch_memory=4.28GiB memory_usage_post_profile=4.02GiB non_torch_memory=1.13GiB kv_cache_size=5.07GiB gpu_memory_utilization=0.90
INFO 12-05 21:14:38 gpu_executor.py:113] # GPU blocks: 11856, # CPU blocks: 9362
INFO 12-05 21:14:38 gpu_executor.py:117] Maximum concurrency for 4096 tokens per request: 46.31x
INFO 12-05 21:14:40 model_runner.py:1400] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 12-05 21:14:40 model_runner.py:1404] If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 1

In [10]:
prompts = [
    "the square root of 10 is",
    "what does a**2 + b**2 equals to according to pythagoras?",
    "What is elliptic curve?",
    "What is prime number?",
]
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

outputs = llm.generate(prompts, sampling_params)
outputs

Processed prompts: 100%|██████████| 4/4 [00:00<00:00, 15.80it/s, est. speed input: 142.20 toks/s, output: 252.80 toks/s]


[RequestOutput(request_id=12, prompt='the square root of 10 is', prompt_token_ids=[1782, 9334, 3704, 315, 220, 16, 15, 374], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=' a rational number**\n\nTo determine if the square root of 10 is a', token_ids=(264, 24438, 1372, 56177, 1249, 8253, 421, 279, 9334, 3704, 315, 220, 16, 15, 374, 264), cumulative_logprob=None, logprobs=None, finish_reason=length, stop_reason=None)], finished=True, metrics=RequestMetrics(arrival_time=1733408470.6773493, last_token_time=1733408470.6773493, first_scheduled_time=1733408470.6788874, first_token_time=1733408470.7134612, time_in_queue=0.0015380382537841797, finished_time=1733408470.919343, scheduler_time=0.001847956000801787, model_forward_time=None, model_execute_time=None), lora_request=None, num_cached_tokens=0),
 RequestOutput(request_id=13, prompt='what does a**2 + b**2 equals to according to pythagoras?', prompt_token_ids=[12555, 1558

In [11]:
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Prompt: 'the square root of 10 is', Generated text: ' a rational number**\n\nTo determine if the square root of 10 is a'
Prompt: 'what does a**2 + b**2 equals to according to pythagoras?', Generated text: ' According to the Pythagorean theorem, in a right-angled triangle, the'
Prompt: 'What is elliptic curve?', Generated text: ' Elliptic curves are algebraic structures defined over a field, which can be'
Prompt: 'What is prime number?', Generated text: ' A prime number is a natural number greater than 1 that has no positive div'
